In [1]:
import datetime
import time
import json

import psycopg2

In [2]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])
    
def process_rdd(rdd):
    # Collect the tuples inside the RDD as a list
    data = rdd.collect()
    # Process the data
    for row in data:
        insertToSQL(row)
        

def insertToSQL(data):
    
    conn = psycopg2.connect(database="bigdatatest",
     host="localhost",
     user="geraldakbar",
     password='bigdata'
     )

    cur = conn.cursor()
    
    social_media = data[0]
    timestamp = data[1]
    count = data[2]
    uniquecount = data[3]

    query = f"""INSERT INTO social_media_data (social_media, timestamp, count, uniquecount)
                    VALUES ('{social_media}', '{timestamp}', {count}, {uniquecount})
                    ON CONFLICT (social_media, timestamp) DO UPDATE
                    SET count = CASE WHEN social_media_data.count IS NULL THEN 1 ELSE social_media_data.count + {count} END,
                    uniquecount = CASE WHEN social_media_data.uniquecount IS NULL THEN 1 ELSE social_media_data.uniquecount + {uniquecount} END,
                    updated_at = NOW();"""


    cur.execute(query)
    print("Inserted Data:",data)
    conn.commit()
    cur.close()
    conn.close()

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc,1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, ['youtube'],
                                      {"metadata.broker.list": "localhost:9092"})

def getYT(lines):
    # map the lines to a key-value pair
    def map(line):
        snippet = line['snippet']
        date = snippet['publishedAt'][0:10] + ' ' + normalizeTime(snippet['publishedAt'][11:19])
        return (("youtube", date), 1)
    
    def userMap(line):
        snippet = line['snippet']
        user = None
        # check if comment is a reply
        if('channelTitle' in snippet):
            user = snippet['channelId']
        else:
            user = snippet['authorChannelId']['value']
        date = snippet['publishedAt'][0:10] + ' ' + normalizeTime(snippet['publishedAt'][11:19])
        return (("youtube", date, user), 1)

    lines = lines.window(5, 5)
    lines = lines.map(lambda x: json.loads(x[1]))
    ytPost = lines.map(map)
    ytPost = ytPost.reduceByKey(lambda x, y: x + y)

    user = lines.map(userMap)
    user = user.transform(lambda rdd: rdd.distinct())
    user = user.map(lambda x: ((x[0][0], x[0][1]), x[1]))
    user = user.reduceByKey(lambda x, y: x + y)

    ytPost = ytPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
    ytPost = ytPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))
    # ytPost = masukin ke postgre
    return ytPost

result = getYT(lines)
# Print
result.foreachRDD(process_rdd)
ssc.start()
ssc.awaitTermination()

23/04/18 06:17:55 WARN streaming.CheckpointWriter: Error in attempt 1 of writing checkpoint to 'hdfs://localhost:9000/user/geraldakbar/checkpoint/checkpoint-1681798675000'
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16506): File does not exist. Holder DFSClient_NONMAPREDUCE_-435975537_16 does not have any open files.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3478)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3445)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:784)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:536)
	at org.apache.hadoop.hdfs.protocol.proto

[Stage 0:>                  (0 + 4) / 5][Stage 2:>                  (0 + 0) / 5]23/04/18 06:17:56 WARN streaming.CheckpointWriter: Error in attempt 1 of writing checkpoint to 'hdfs://localhost:9000/user/geraldakbar/checkpoint/checkpoint-1681798676000'
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16511): File does not exist. Holder DFSClient_NONMAPREDUCE_-435975537_16 does not have any open files.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3478)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3445)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:784)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodePr

23/04/18 06:17:58 WARN streaming.CheckpointWriter: Error in attempt 1 of writing checkpoint to 'hdfs://localhost:9000/user/geraldakbar/checkpoint/checkpoint-1681798678000'
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16519): File does not exist. Holder DFSClient_NONMAPREDUCE_-435975537_16 does not have any open files.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3478)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3445)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:784)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:536)
	at org.apache.hadoop.hdfs.protocol.proto

Inserted Data: ('youtube', '2023-04-04 16:45:00', 1, 1)
Inserted Data: ('youtube', '2023-04-04 17:05:00', 2, 2)


23/04/18 06:18:00 WARN hdfs.DFSClient: DataStreamer Exception
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16530): File does not exist. [Lease.  Holder: DFSClient_NONMAPREDUCE_-435975537_16, pendingcreates: 1]
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.analyzeFileState(FSNamesystem.java:3185)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getAdditionalBlock(FSNamesystem.java:3032)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.addBlock(NameNodeRpcServer.java:722)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.addBlock(ClientNamenodeProtocolServerSideTranslatorPB.java:492)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)

...

Inserted Data: ('youtube', '2023-04-04 17:00:00', 1, 1)
Inserted Data: ('youtube', '2023-04-04 17:05:00', 1, 1)


23/04/18 06:18:10 WARN streaming.CheckpointWriter: Error in attempt 1 of writing checkpoint to 'hdfs://localhost:9000/user/geraldakbar/checkpoint/checkpoint-1681798690000'
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16578): File does not exist. [Lease.  Holder: DFSClient_NONMAPREDUCE_-435975537_16, pendingcreates: 1]
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3478)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3445)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:784)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:536)
	at org.apache.hadoop.hdfs.protocol.proto

23/04/18 06:18:12 WARN streaming.CheckpointWriter: Error in attempt 1 of writing checkpoint to 'hdfs://localhost:9000/user/geraldakbar/checkpoint/checkpoint-1681798692000'
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16590): File does not exist. [Lease.  Holder: DFSClient_NONMAPREDUCE_-435975537_16, pendingcreates: 1]
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3478)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3445)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:784)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:536)
	at org.apache.hadoop.hdfs.protocol.proto

23/04/18 06:18:13 WARN streaming.CheckpointWriter: Error in attempt 1 of writing checkpoint to 'hdfs://localhost:9000/user/geraldakbar/checkpoint/checkpoint-1681798693000'
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/geraldakbar/checkpoint/temp (inode 16597): File does not exist. [Lease.  Holder: DFSClient_NONMAPREDUCE_-435975537_16, pendingcreates: 1]
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3386)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3478)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3445)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:784)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:536)
	at org.apache.hadoop.hdfs.protocol.proto

KeyboardInterrupt: 